In [4]:
import cv2
import torch
import numpy as np
import os
import glob
import subprocess
from google.colab.patches import cv2_imshow
from PIL import Image
import torchvision.transforms as transforms
from torchvision import models
import torch.nn.functional as F

# Tikriname ir įdiegiame trūkstamas bibliotekas
required_packages = ["ultralytics", "deep_sort_realtime"]
for package in required_packages:
    try:
        __import__(package)
    except ModuleNotFoundError:
        print(f"📦 Diegiama: {package}")
        try:
            subprocess.check_call(["pip", "install", package])
        except subprocess.CalledProcessError as e:
            print(f"❌ Klaida diegiant {package}: {e}")
            exit()

import ultralytics
import deep_sort_realtime

from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Nustatome katalogus
video_directory = "input_videos"
sample_directory = "car"
output_directory = "output_videos"
os.makedirs(output_directory, exist_ok=True)

# Naudojame MobileNetV2 vietoj ResNet18 greitesniam veikimui
vehicle_model = models.mobilenet_v2(pretrained=True)
vehicle_model.eval()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features(image):
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        features = vehicle_model(image)
    return features

# Įkeliame pateiktas automobilio pavyzdžių nuotraukas
sample_images = []
sample_features = []
for img_path in glob.glob(os.path.join(sample_directory, "*.jpg")):
    img = cv2.imread(img_path)
    sample_images.append(img)
    sample_features.append(extract_features(img))

# YOLO modelis automobilio aptikimui
model = YOLO("yolov8n.pt")  # Optimizuota nano versija greitesniam veikimui

# DeepSORT ReID modelis sekimui
tracker = DeepSort(max_age=50, max_iou_distance=0.4)  # Pagerintas stabilumas

# Apdorosime visus vaizdo įrašus
video_files = sorted(glob.glob(os.path.join(video_directory, "*.mp4")))
for video_path in video_files:
    print(f"🔄 Apdorojamas vaizdo įrašas: {video_path}")

    # Vaizdo įrašo įkėlimas
    capture = cv2.VideoCapture(video_path)
    width = int(capture.get(3))
    height = int(capture.get(4))
    fps = int(capture.get(cv2.CAP_PROP_FPS))

    output_filename = os.path.join(output_directory, os.path.basename(video_path))
    out = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    tracked_cars = {}
    target_car_id = None
    best_match_threshold = 0.6  # Slenkstis, kad filtruotų klaidingus atitikmenis

    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            break

        # Aptinkame automobilius su YOLOv8
        results = model(frame)[0]
        detections = []
        best_match_score = float('inf')  # L2 atstumas, mažesnis geresnis
        best_bbox = None

        for result in results.boxes.data:
            x1, y1, x2, y2, conf, cls = result.tolist()
            if int(cls) == 2 or int(cls) == 7:
                cropped_car = frame[int(y1):int(y2), int(x1):int(x2)]
                car_features = extract_features(cropped_car)
                match_scores = [torch.dist(car_features, sample_feat).item() for sample_feat in sample_features]
                cosine_scores = [F.cosine_similarity(car_features, sample_feat).item() for sample_feat in sample_features]

                min_distance = min(match_scores)
                max_similarity = max(cosine_scores)

                if min_distance < best_match_score and max_similarity > best_match_threshold:
                    best_match_score = min_distance
                    best_bbox = (x1, y1, x2, y2)

        if best_bbox:
            detections.append((best_bbox, 1.0, 'car'))

        # Atlikti sekimą su DeepSORT
        tracker_outputs = tracker.update_tracks(detections, frame=frame)

        for track in tracker_outputs:
            if track.is_confirmed():
                x1, y1, x2, y2 = track.to_tlbr()
                track_id = track.track_id

                if target_car_id is None:
                    target_car_id = track_id

                if track_id == target_car_id:
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 3)
                    cv2.putText(frame, f'Sekamas automobilis {track_id}', (int(x1), int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                    print(f"🚗 Sekamas automobilis ID: {track_id} ties ({x1}, {y1}, {x2}, {y2})")

        out.write(frame)

    capture.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"✅ Vaizdo įrašas išsaugotas: {output_filename}")



🔄 Apdorojamas vaizdo įrašas: input_videos/input video.mp4

0: 384x640 10 cars, 1 bus, 3 trucks, 145.1ms
Speed: 4.9ms preprocess, 145.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 152.8ms
Speed: 5.4ms preprocess, 152.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 3 trucks, 144.3ms
Speed: 3.7ms preprocess, 144.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
🚗 Sekamas automobilis ID: 1 ties (66.92106182299983, 1522.8868996754022, 742.1568435541634, 3434.6680303520075)

0: 384x640 9 cars, 1 bus, 4 trucks, 163.2ms
Speed: 3.7ms preprocess, 163.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
🚗 Sekamas automobilis ID: 1 ties (28.753974428265963, 1535.089868392657, 706.4225420029336, 3460.8856458620585)

0: 384x640 8 cars, 3 trucks, 145.9ms
Speed: 7.3ms preprocess, 145.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
🚗 Sekamas auto